In [1]:
import fiona
import pandas as pd
import geopandas
import os

In [2]:
os.chdir('/Users/dannyha/SoCalEVCSNetwork')
os.getcwd()

'/Users/dannyha/SoCalEVCSNetwork'

# Data Importing and Cleaning

## SoCal Zipcodes and Geometry

In [3]:
socal_zipcodes = pd.read_csv (r'data/socal_zipcodes/socal_zip.csv')

In [4]:
socal_zipcodes = socal_zipcodes[['ZCTA5CE10','GEOID10','INTPTLAT10','INTPTLON10','geometry']]

In [5]:
socal_zipcodes = socal_zipcodes.rename(columns={"ZCTA5CE10": "Zip Code", "GEOID10": "GEOID", "INTPTLAT10": "Latitude",
                              "INTPTLON10": "Longitude", "geometry": "Geometry"})

In [6]:
socal_zipcodes.head()

,Zip Code,GEOID,Latitude,Longitude,Geometry
0,91504,691504,34.204568,-118.326365,"POLYGON ((-118.348583 34.208461, -118.348564 3..."
1,91606,691606,34.185767,-118.388324,"POLYGON ((-118.403066 34.19394, -118.402564 34..."
2,90640,690640,34.015444,-118.111012,"POLYGON ((-118.128344 33.985893, -118.127359 3..."
3,91770,691770,34.065493,-118.084529,"POLYGON ((-118.107291 34.066222, -118.107323 3..."
4,90290,690290,34.107728,-118.615981,"POLYGON ((-118.640641 34.125821, -118.640133 3..."


In [7]:
socal_zipcodes['Zip Code'] = socal_zipcodes['Zip Code'].apply(str)

## DMV Data

In [100]:
vehicles = pd.read_csv (r'data/vehicle-count-as-of-1-1-2020.csv',header=1)

/Users/dannyha/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [101]:
#filtering by socal zipcodes
socal_vehicles = vehicles[vehicles['Zip Code'].isin(socal_zipcodes['Zip Code'])]

In [102]:
#filtering light duty vehicles
socal_vehicles = socal_vehicles[socal_vehicles['Duty']=='Light']

In [103]:
socal_vehicles.head()

,Date,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
0,1/1/2020,90001,2007,Gasoline,ACURA,Light,15
1,1/1/2020,90002,2007,Gasoline,ACURA,Light,20
2,1/1/2020,90003,2007,Gasoline,ACURA,Light,29
3,1/1/2020,90004,2007,Gasoline,ACURA,Light,19
4,1/1/2020,90006,2007,Gasoline,ACURA,Light,15


In [104]:
socal_vehicles.dtypes

Date          object
Zip Code      object
Model Year    object
Fuel          object
Make          object
Duty          object
Vehicles      object
dtype: object

In [105]:
#filtering EVs and Light-duty vehicles
socal_evs = socal_vehicles[socal_vehicles['Fuel'].isin(['Battery Electric','Plug-in Hybrid'])]
socal_evs = socal_evs.reset_index()
socal_evs = socal_evs[['Zip Code','Model Year','Fuel','Make','Duty','Vehicles']]

In [106]:
socal_evs

,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
0,90013,2008,Battery Electric,OTHER/UNK,Light,1
1,90023,2008,Battery Electric,OTHER/UNK,Light,1
2,90024,2008,Battery Electric,OTHER/UNK,Light,3
3,90027,2008,Battery Electric,OTHER/UNK,Light,1
4,90049,2008,Battery Electric,OTHER/UNK,Light,3
...,...,...,...,...,...,...
20905,92629,<2007,Battery Electric,OTHER/UNK,Light,2
20906,92663,<2007,Battery Electric,OTHER/UNK,Light,2
20907,92805,<2007,Battery Electric,OTHER/UNK,Light,1
20908,92835,<2007,Battery Electric,OTHER/UNK,Light,1


In [107]:
socal_vehicles['Vehicles'] = socal_vehicles['Vehicles'].astype(int)

In [108]:
socal_vehicles.dtypes

Date          object
Zip Code      object
Model Year    object
Fuel          object
Make          object
Duty          object
Vehicles       int64
dtype: object

In [109]:
socal_evs_agg = socal_evs[['Zip Code','Vehicles']].groupby('Zip Code').sum('Vehicles')
socal_evs_agg

,Vehicles
Zip Code,
90001,134.0
90002,162.0
90003,136.0
90004,1294.0
90005,560.0
...,...
93543,60.0
93550,NaN
93551,1074.0


In [110]:
socal_evs_agg['Electric Vehicles'] = socal_evs_agg['Vehicles']
socal_evs_agg = socal_evs_agg.drop(columns='Vehicles') 
socal_evs_agg.head()

,Electric Vehicles
Zip Code,
90001,134.0
90002,162.0
90003,136.0
90004,1294.0
90005,560.0


In [113]:
socal_vehicles_by_zip = socal_vehicles[['Zip Code','Vehicles']].groupby('Zip Code').sum('Vehicles')
socal_vehicles_by_zip

,Vehicles
Zip Code,
90001,67498
90002,61550
90003,82098
90004,66370
90005,34626
...,...
93543,21824
93550,105820
93551,86680


In [115]:
ev_percentage = socal_vehicles_by_zip.merge(socal_evs_agg, how='inner', on='Zip Code')
ev_percentage.columns=['Total Vehicles','Electric Vehicles']
ev_percentage.head()

,Total Vehicles,Electric Vehicles
Zip Code,,
90001,67498,134.0
90002,61550,162.0
90003,82098,136.0
90004,66370,1294.0
90005,34626,560.0


In [116]:
ev_percentage['EV Percentage'] = ev_percentage['Electric Vehicles'] / ev_percentage['Total Vehicles']
ev_percentage = ev_percentage[['Electric Vehicles','Total Vehicles','EV Percentage']]
ev_percentage.head()

,Electric Vehicles,Total Vehicles,EV Percentage
Zip Code,,,
90001,134.0,67498,0.001985
90002,162.0,61550,0.002632
90003,136.0,82098,0.001657
90004,1294.0,66370,0.019497
90005,560.0,34626,0.016173


In [117]:
merged = ev_percentage.merge(socal_zipcodes, how='left', on='Zip Code')
merged

,Zip Code,Electric Vehicles,Total Vehicles,EV Percentage,GEOID,Latitude,Longitude,Geometry
0,90001,134.0,67498,0.001985,690001,33.974027,-118.249509,"POLYGON ((-118.237949 33.96015, -118.238527 33..."
1,90002,162.0,61550,0.002632,690002,33.949099,-118.246737,"POLYGON ((-118.23737 33.958521, -118.237377 33..."
2,90003,136.0,82098,0.001657,690003,33.964131,-118.272783,"POLYGON ((-118.282698 33.964166, -118.282698 3..."
3,90004,1294.0,66370,0.019497,690004,34.076198,-118.310723,"POLYGON ((-118.311601 34.06895799999999, -118...."
4,90005,560.0,34626,0.016173,690005,34.059163,-118.306892,"MULTIPOLYGON (((-118.302852 34.062356, -118.30..."
...,...,...,...,...,...,...,...,...
346,93543,60.0,21824,0.002749,693543,34.481108,-117.982006,"POLYGON ((-117.938869 34.470802, -117.939103 3..."
347,93550,NaN,105820,NaN,693550,34.408383,-118.123597,"POLYGON ((-118.189699 34.37524, -118.189689 34..."
348,93551,1074.0,86680,0.012390,693551,34.604110,-118.239005,"POLYGON ((-118.201212 34.630971, -118.199343 3..."
349,93552,330.0,59876,0.005511,693552,34.570867,-118.023744,"POLYGON ((-118.058392 34.583265, -118.058417 3..."
